In [8]:
import sys
import os
import pandas as pd
import numpy as np
# import pandas_profiling
import json
import math
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',1000)
import sklearn
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split 
from collections import Counter
from pandas.api.types import is_dict_like
import itertools
import time
import matplotlib.pyplot as plt
sys.path.append('/Users/yantingting/PycharmProjects/modeling')
import plot_tools as pl
import metrics
import database_conncet as data_con
import summary_statistics as ss
import feature_selection as fs
from data_io_utils import *
import data_processing as dp
from jinja2 import Template

In [9]:
file_path = '/Users/yantingting/Seafile/风控/模型/14 巴西/Portocred/data/'

# 数据拼接

In [10]:
var_bank_data = load_data_from_pickle(file_path,'bank_var.pkl')
var_bank_data.head()
var_app_freq = pd.read_csv(file_path + 'var_app_freq.csv')
var_app_tfidf = pd.read_csv(file_path + 'var_app_tfidf.csv')
var_contact = pd.read_csv(file_path + 'var_contact.csv', dtype = {'loan_id': int})
var_deviceinfo = pd.read_csv(file_path +'var_deviceinfo.csv')
var_othersdk = pd.read_csv(file_path + 'var_othersdk.csv')
flag_df = pd.read_csv(os.path.join(file_path,'flag_df.csv'))[['loan_id','sample_flag']]
# flag_df['loan_id'] = flag_df['loan_id'].astype(str)
df_sdk = pd.merge(var_bank_data, var_app_freq, on = 'loan_id', how = 'left')
df_sdk = pd.merge(df_sdk, var_app_tfidf, on = 'loan_id', how = 'left')
df_sdk = pd.merge(df_sdk, var_contact, on = 'loan_id', how = 'left')
df_sdk = pd.merge(df_sdk, var_deviceinfo, on = 'loan_id', how = 'left')
df_sdk = pd.merge(df_sdk, var_othersdk, on = 'loan_id', how = 'left')
df_sdk =pd.merge(df_sdk, flag_df, on = 'loan_id', how = 'left')
save_data_to_pickle(df_sdk,file_path,'df_var_all.pkl')
df_sdk.shape
df_sdk['sample_flag'].value_counts()


,loan_id,date,fpd_1,fpd_2,fpd_3,serasa_score_y,boa_vista_(bvs)_score_y,internal_score_y,age,social_status,active_companies,serasa_score,boa_vista_score,internal_score,income,irs_status_FALHA,irs_status_SEGUE,bank_number_0341,bank_number_0237,bank_number_0104,bank_number_0033,bank_number_0001,bank_number_0041,bank_number_0756,bank_number_0655,bank_number_0021,bank_number_0077,bank_number_0748,bank_number_0037,bank_number_0070,bank_number_0151,occupation_Assalariados,occupation_FuncionÃ¡rios PÃºblicos,occupation_Aposentados e Pensionistas,occupation_Capitalista rend aplic / aluguel,gender_M,gender_F,education_degree_5,education_degree_7,education_degree_6,education_degree_3,education_degree_8,education_degree_4,education_degree_2,state_name_SP,state_name_RJ,state_name_MG,state_name_RS,state_name_SC,state_name_BA,state_name_PR,state_name_DF,state_name_GO,state_name_CE,state_name_PA,state_name_PE,state_name_MT,state_name_PI,state_name_TO,state_name_AM,state_name_MA,state_name_RN,state_name_ES,state_name_AC,state_name_PB,state_name_AL,state_name_MS,state_name_AP,state_name_SE,state_name_RO,maritial_status_S,maritial_status_C,maritial_status_D,maritial_status_P,maritial_status_O,maritial_status_V,maritial_status_J,email1_gmail,email1_hotmail,email1_yahoo,email1_outlook,email1_bol,email1_live,email1_terra,email1_uol,email1_oi,email1_agenpol
0,3808899799,20191017,1,0,0,1,0,0,41,1,0,582.0,NaN,NaN,1500.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,3808916175,20191017,1,1,0,1,0,0,45,1,1,588.0,NaN,NaN,2680.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,3808778599,20191014,1,0,0,1,0,0,39,0,0,567.0,NaN,NaN,1324.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,3808862384,20191014,1,0,0,1,0,0,28,0,0,578.0,NaN,NaN,1603.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
4,3808855137,20191014,0,0,0,1,0,0,53,1,0,667.0,NaN,NaN,2800.0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


(445, 449)

0    333
1    112
Name: sample_flag, dtype: int64

# 字典拼接

In [11]:
dict_bank = pd.read_csv(file_path+'var_dict_bank.csv')
# dict_bank.shape
dict_app_freq = pd.read_csv(file_path + 'var_dict_app_freq.csv')
# dict_app_freq.shape
dict_app_tfidf = pd.read_csv(file_path + 'var_dict_app_tfidf.csv')
# dict_app_tfidf.shape
dict_contact = pd.read_csv(file_path + 'var_dict_contact.csv')
# dict_contact.shape
dict_deviceinfo = pd.read_csv(file_path +'var_dict_deviceinfo.csv')
# dict_deviceinfo.shape
dict_othersdk = pd.read_csv(file_path + 'var_dict_othersdk.csv')
# dict_othersdk.shape
dict_all = pd.concat([dict_bank,dict_app_freq,dict_app_tfidf,dict_contact,dict_deviceinfo,dict_othersdk],axis = 0)
dict_all.to_csv(os.path.join(file_path,'dict_all.csv'), index = False)
dict_all.shape

(443, 4)